# Creating a data-backed playing field
Felix Zaussinger | 30.10.2020

## Core Analysis Goal(s)

1. Find LULC GIS layer and download data.
2. Downsampling to ~ 3 lower resolutions/larger cellsizes via rasterio.

## Key Insight(s)

1. The Historical and present land use dataset (downloaded 30.11.2020 at https://hlamap.org.uk/) useful for our purposes. The only thing that is still missing are forests.
2.
3.

## Arising Todo(s)

1. Find forest inventory data and merge with HLA data to get the full picture.

In [ ]:
import os
import sys
import logging

import numpy as np
import scipy as sp
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import geopandas as gpd
import rasterio as rio

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

Promising data sources for GIS layers are summarised here: https://www.environment.gov.scot/maps/other-map-tools/#land.

These look really cool:

- HabMoS EUNIS Land Cover Scotland (https://map.environment.gov.scot/sewebmap/?layers=eunisLandCoverScotland&extent=-787366,74257,1322365,1708636)
- EUNIS Combined Map (https://hub.jncc.gov.uk/assets/2048c042-5d68-46c6-8021-31d177b00ac4)
- Benchmark LULC data set ( not for free :( ): https://www.ceh.ac.uk/services/land-cover-map-2015
- National Biodiversity Network: https://records.nbnatlas.org/explore/your-area#52.9548|1.1580999999999904|12|ALL_SPECIES
- Historical and present land use (downloaded 30.11.2020): https://hlamap.org.uk/
- Forest inventory and grant eligibility data: https://open-data-scottishforestry.hub.arcgis.com/search?collection=Dataset
- Biosphere Reserves: https://gateway.snh.gov.uk/natural-spaces/dataset.jsp?dsid=BIOSPH
- Geodata in general: https://www.spatialdata.gov.scot/geonetwork/srv/eng/catalog.search#/home

The Historic Land-use Assessment project (HLA) is digitally recording land use across Scotland. It maps both activities like industrial farming or ski areas that are current today, and also land use activities from periods in the past, such as charcoal burning or prehistoric agriculture and settlement. The data has been gathered since 1997 as a partnership between Historic Scotland (HS) and the Royal Commision on the Ancient and Historical Monuments of Scotland and coverage was completed in 2015, a  year that also saw the merging of the two organisations into Historic Environment Scotland.

In [ ]:
data_raw = os.path.join("..", "data", "raw")
data_processed = os.path.join("..", "data", "processed")
lulc_fpath = os.path.join(data_raw, "HLAmap", "HLA_Dec2017.shp")

lulc = gpd.read_file(lulc_fpath)
lulc

In [ ]:
lulc.Period.unique()

In [ ]:
lulc.Historic_L.unique()

In [ ]:
lulc.crs

In [ ]:
lulc.plot(column="Type")

Subset Galloway and Southern Ayrshire Biosphere Reserve

In [ ]:
parks_fpath = os.path.join(data_raw, "BIOSPH_SCOTLAND_ESRI", "BIOSPH_SCOTLAND.shp")
# lulc_park = gpd.clip(lulc, park_polygon)
parks = gpd.read_file(parks_fpath)

In [ ]:
parks.crs

In [ ]:
parks.crs == lulc.crs

In [ ]:
our_park = parks.loc[parks.NAME == parks.NAME.unique()[0]]

In [ ]:
our_park.plot()

In [ ]:
our_park_polygon = our_park.dissolve(by="NAME")
our_park_polygon.plot()

Decision based on https://hlamap.org.uk/pdfs/HLA%20Glossary.pdf

In [ ]:
lulc_modern = lulc.loc[lulc.Period == "Late 20th Century-Present"]
lulc_modern_valid = lulc_modern.loc[lulc_modern.is_valid]
lulc_modern_valid

In [ ]:
park_modern_lulc = gpd.clip(lulc_modern_valid, our_park_polygon.convex_hull.envelope)

Based on https://stackoverflow.com/questions/54562069/multi-layer-gdb-files-in-python/54563846

In [ ]:
# import fiona
# eunis_fpath = os.path.join(data_raw, "C20191101_EUNIS_L3_CombinedMap.gdb")
# eunis = gpd.read_file(eunis_fpath, layer=fiona.listlayers(eunis_fpath)[0])

In [ ]:
park_modern_lulc.columns

In [ ]:
ax = our_park.plot()
park_modern_lulc.plot(
    column = "Type",
    legend = True,
    ax=ax
)

In [ ]:
park_modern_lulc.Type.unique()

In [ ]:
park_modern_lulc["lulc_type_num"] = 255

https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html

In [ ]:
mapping = {
    'Country Park': 1,
    'Motorway and Major Roads': 2,
    'Landfill Site': 3,
    'Opencast Site': 4,
    'Industrial-scale Farming Unit': 5,
    'Restored Agricultural Land': 6,
    'Unenclosed Improved Pasture': 7,
    'Deer Lawn': 8,
    'Power Generation': 9,
    'Rough Grazing': 10
}

In [ ]:
park_modern_lulc["lulc_type_num"] = [mapping[k] for k in park_modern_lulc.Type.values]

In [ ]:
park_modern_lulc["lulc_type_num"] = park_modern_lulc.lulc_type_num.astype("int16")


In [ ]:
park_modern_lulc.lulc_type_num

In [ ]:
import rasterio
from rasterio import features

# raster dims
rows = 100
cols = 100

data_processed = os.path.join("..", "data", "processed")
out_fn = os.path.join(data_processed, 'HLA_rasterized_{}_{}.tif'.format(rows, cols))

with rasterio.open(out_fn, 'w+', driver='GTiff', width=cols, height=rows, count=1, dtype=np.int16, nodata=255) as out:
    out_arr = out.read(1)
    print(out_arr)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(park_modern_lulc.geometry, park_modern_lulc.lulc_type_num.values))
    burned = features.rasterize(shapes=shapes, fill=255, out=out_arr, all_touched=False)
    out.write_band(1, burned)

np.unique(burned)